In [1]:
import polars as pl
import silverfund.data_access_layer as dal
from datetime import date

In [2]:
start_date = date(2024, 7, 31)
end_date = date(2024, 12, 31)

In [3]:
df = pl.read_parquet("research/constraints/results/full_investment_bt.parquet")

df = df.sort(["date", "barrid"])

df

date,barrid,weight,fwd_ret
date,str,f64,f64
1995-07-31,"""USAA191""",-0.000338,-0.014814
1995-07-31,"""USAA1W1""",0.000068,-0.006849
1995-07-31,"""USAA1Y1""",0.000638,0.000001
1995-07-31,"""USAA2L1""",0.000355,0.057472
1995-07-31,"""USAA311""",-0.000221,-0.019869
…,…,…,…
2024-11-29,"""USBQFF1""",0.000336,null
2024-11-29,"""USBQGD1""",0.000465,null
2024-11-29,"""USBQLB1""",-0.00031,null


In [4]:
exposures = dal.load_factor_exposures(start_date=start_date, end_date=end_date)

exposures

Loading Barra factor exposures: 100%|██████████| 1/1 [00:05<00:00,  5.30s/it]


barrid,factor,date,exposure
str,str,date,f64
"""ARGA1B1""","""BETA""",2024-07-31,1.029
"""ARGA1B1""","""BETA""",2024-08-01,1.049
"""ARGA1B1""","""BETA""",2024-08-02,2.198
"""ARGA1B1""","""BETA""",2024-08-05,2.238
"""ARGA1B1""","""BETA""",2024-08-06,2.159
…,…,…,…
"""USBQWH1""","""VALUE""",2024-12-24,null
"""USBQWH1""","""VALUE""",2024-12-26,null
"""USBQWH1""","""VALUE""",2024-12-27,0.856


In [5]:
exposures.filter(pl.col("barrid").eq("USBQOR1")).group_by("date").agg(
    pl.col("exposure").sum()
).sort("date").filter(pl.col("exposure").ne(0))

date,exposure
date,f64
2024-10-29,-1.958
2024-10-30,-1.926
2024-10-31,-0.379
2024-11-01,-0.494
2024-11-04,-0.428
…,…
2024-12-24,2.619
2024-12-26,2.52
2024-12-27,2.292


In [6]:
fac = pl.read_csv("daily_factor_returns.csv")

fac

date,BETA,DIVYILD,EARNQLTY,EARNYILD,GROWTH,LEVERAGE,LIQUIDTY,LTREVRSL,MGMTQLTY,MIDCAP,MOMENTUM,PROFIT,PROSPECT,RESVOL,SIZE,VALUE,OILGSDRL,OILGSEQP,OILGSEXP,OILGSCON,CHEM,SPTYCHEM,CNSTMATL,CONTAINR,PAPER,ALUMSTEL,PRECMTLS,AERODEF,CNSTMACH,INDMACH,BLDGPROD,TRADECO,CNSTENG,ELECEQP,CONGLOM,COMSVCS,…,HOUSEDUR,HOMEBLDG,LEISPROD,LEISSVCS,RESTAUR,MEDIA,DISTRIB,NETRET,APPAREL,SPTYSTOR,SPLTYRET,PSNLPROD,FOODRET,FOODPROD,BEVTOB,HLTHEQP,MGDHLTH,HLTHSVCS,BIOLIFE,PHARMA,BANKS,DIVFIN,LIFEINS,INSURNCE,REALEST,SEMIEQP,SEMICOND,INTERNET,SOFTWARE,COMMEQP,COMPELEC,TELECOM,WIRELESS,ELECUTIL,GASUTIL,MULTUTIL,COUNTRY
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1995-06-30""",0.001561,-0.000111,-0.001143,0.001047,0.000158,-0.001521,-0.001092,-0.000133,0.000511,0.001629,0.000509,-0.001739,0.00018,0.000732,-0.002761,-0.000752,-0.001353,-0.00814,-0.001717,-0.006427,0.00103,-0.001033,0.002864,-0.002957,0.012288,0.011518,-0.017056,0.000148,0.006751,0.005296,0.000959,0.000154,0.01676,0.008939,0.006313,0.004885,…,0.00952,0.001529,0.004104,0.000092,-0.002673,0.004194,0.011079,0.0172721,0.002881,0.008131,0.003657,-0.003212,-0.00081,0.001588,0.003202,0.001484,-0.000214,0.004634,0.003599,-0.003253,-0.006071,-0.004603,-0.006194,-0.006373,0.004951,-0.001164,-0.012696,-0.001168,-0.001577,0.000951,-0.000992,0.003418,0.007565,-0.002961,-0.000101,-0.00267,0.002391
"""1995-07-03""",0.000089,0.000245,-0.000241,0.000078,-0.000233,-0.000011,0.000345,-0.000387,0.000382,-0.001348,0.000186,0.000121,0.000044,0.000846,0.000553,-0.000595,-0.008023,-0.001949,-0.00524,0.003236,-0.00041,-0.003119,-0.004636,0.00586,0.002184,0.016423,0.000831,-0.005342,0.000774,-0.000042,-0.003573,0.002709,-0.001686,-0.002155,-0.001677,0.001693,…,0.002571,-0.007797,0.004425,-0.001245,-0.003406,-0.001094,-0.003775,0.002584,0.003749,0.002998,0.00315,-0.0012,-0.002144,0.000817,-0.000368,-0.001064,0.010634,0.003074,-0.007351,-0.000182,0.000164,-0.000127,0.002304,-0.003008,-0.001202,-0.007386,-0.003535,0.000264,-0.00341,0.003353,0.003286,0.000568,0.000633,-0.002092,0.000127,-0.000296,0.003053
"""1995-07-05""",0.002074,-0.000567,-0.000175,0.000684,-0.000317,0.000955,0.001143,-0.00159,-0.00045,0.000674,0.001383,-0.001668,0.000101,0.001207,-0.001777,0.000222,-0.00127,-0.006295,-0.009669,-0.00724,0.009939,0.00285,-0.004074,-0.006006,0.011728,0.02883,0.012252,-0.000353,0.018447,0.006424,0.005284,0.005781,-0.002727,0.001254,0.011716,-0.000077,…,0.000587,-0.00863,0.000791,-0.004699,0.008095,-0.00024,0.005592,-0.002774,0.010066,0.00198,0.010819,0.002838,0.00827,-0.005234,-0.005646,-0.001478,-0.008571,-0.002187,0.004604,-0.00407,-0.004594,-0.003743,-0.005177,-0.001046,-0.002685,0.01328,-0.006522,0.011676,-0.000352,0.003096,0.004514,-0.003579,0.021098,-0.003109,-0.009164,-0.001788,0.001546
"""1995-07-06""",0.002513,-0.000337,-0.000736,-0.000194,0.000978,0.001209,0.000092,-0.001845,0.000933,0.000031,0.000601,0.000279,0.000528,-0.000909,0.001494,0.000203,-0.00609,-0.001388,-0.003006,0.001486,0.001738,0.000873,0.006848,0.006944,-0.002223,-0.001499,0.026613,-0.001346,0.010105,0.003085,0.00668,-0.002192,0.003364,-0.002968,0.003993,-0.000212,…,0.001798,0.050161,-0.002678,-0.003664,-0.012678,-0.001666,-0.005317,-0.008395,-0.021365,-0.007274,0.011479,-0.014242,0.003142,0.001866,0.001265,-0.00158,-0.017436,-0.008381,0.000839,-0.007166,-0.001606,-0.002697,-0.000886,-0.000173,0.001506,0.020481,0.012209,0.002698,0.002732,0.001522,0.003423,0.008178,0.001505,-0.003184,-0.005876,0.00103,0.011325
"""1995-07-07""",0.003138,-0.000207,-0.000777,-0.000055,-0.000283,0.000313,0.002068,-0.001304,0.000578,0.001948,0.001245,-0.001208,0.000394,0.001758,-0.002841,-0.000434,-0.020638,0.007834,-0.00167,0.001178,0.019815,0.012524,0.004802,0.003252,0.0111183,0.035416,0.01514,0.00265,0.020405,